In [ ]:
import cv2
from matplotlib import pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
from PIL import Image
from PIL import ImageFilter
from matplotlib import animation
from PIL import ImageEnhance
from tkinter import filedialog
import tkinter as tk
import pickle
from os import sys
import matplotlib.pyplot as plt
import argparse
from statistics import mean
import os

from scipy.interpolate import interp1d
from scipy.io import loadmat
from scipy import signal as sig
import pandas as pd
import numpy as np



def FindReferenceSystem2(video_path):
    
    HandleBEHAV = cv2.VideoCapture(video_path, 0)
    length = int(HandleBEHAV.get(cv2.CAP_PROP_FRAME_COUNT))
    
    _ , IMG1 = HandleBEHAV.read()

    linemask2= IMG1[601:602, : , 0]

    linemask2 = np.invert(linemask2)
    linemask2 = linemask2.flatten()

    smoothmask = smooth(linemask2)

    b, a = butter(8, 0.075)
    smoothfiltered = filtfilt(b, a, smoothmask)

    diffmask = np.diff(smoothfiltered,n=1)

    PeaksNO2 , values = sig.find_peaks(diffmask, height = 0.01)

    list1 = []

    if np.size(PeaksNO2) > 1: 

        var_Peaks = 1
        height = 0.01
        TakenHeight = []
        
        while np.size(PeaksNO2) > var_Peaks :
            height = height +0.01
            PeaksNO2_2 , values = sig.find_peaks(diffmask, height)
            
            if np.size(PeaksNO2_2) == 1:
                list1.append(PeaksNO2_2)
                TakenHeight.append(height)
                break
                
    Treshold = (TakenHeight[0])
    Peaks3 = ((list1 [0])  - 15)
    Peaks4 = ((list1 [0])  + 15)
    
    return [Peaks3,Peaks4,Treshold]



def OscObj2 (video_path): 
    
    HandleBEHAV = cv2.VideoCapture(video_path, 0)
    length = int(HandleBEHAV.get(cv2.CAP_PROP_FRAME_COUNT))
    _ , IMG1 = HandleBEHAV.read()
    ListObj3 = []

    for framne_nb in range(length-1) :

        _ , IMG1 = HandleBEHAV.read()

        Check = np.vstack((Check , IMG1[601:602, : , 0]))

        linemask = IMG1[601:602, int(Reference_peaks2[0]):int(Reference_peaks2[1])]

        lineIMG = linemask.copy()

        linemask3 = linemask.flatten()

        smoothmask = smooth(linemask3)

        X = np.arange(0,np.size(smoothmask),1)
        Xinterp = np.arange(0,np.size(smoothmask),0.1)
        f2 = interp1d(X, abs(smoothmask), kind='slinear', fill_value="extrapolate")

        raw_interp = f2(Xinterp)

        interpol_img = np.reshape(raw_interp,(1, np.shape(raw_interp)[0]))
        interpol_img = np.vstack((interpol_img,interpol_img,interpol_img,interpol_img,interpol_img,interpol_img,interpol_img,interpol_img,interpol_img,interpol_img))

        b, a = butter(8, 0.075)
        smoothfiltered = filtfilt(b, a, raw_interp)

        diffmask = np.diff(smoothfiltered,n=1)

        DPeaks , values = sig.find_peaks(abs(diffmask), height = 0.045)
        
        

        if np.size(DPeaks) > 0:
            ListObj3.append(DPeaks[0])

        else :

            ListObj3.append(np.nan)

            
    LineFrame = Check[ 0:np.shape(Check)[0] , int(Reference_peaks2[0]):int(Reference_peaks2[1])]

    #print(np.shape(LineFrame))

    Modif = smooth(LineFrame[0,:])

    InterpFrame = Easyinterp(Modif)

    DiffFrame = np.diff(abs(InterpFrame), n=1)

    listmax = [ np.argmin(DiffFrame) ]


    for I in range(1,np.shape(LineFrame)[0],1):

        Modif = np.vstack( ( Modif, smooth(LineFrame[I,:]) ) )
        InterpFrame = np.vstack( ( InterpFrame , Easyinterp(Modif[I,:] ) ) )
        DiffFrame = np.vstack ( ( DiffFrame , diff(InterpFrame[I,:] ) ) )

        listmax.append(np.argmin(DiffFrame[I,:]))